In [1]:
import re
import json
import os

In [ ]:
def create_json_n1(input_path: str, output_path: str) -> dict:
    """
    Lê um arquivo de texto, extrai blocos de dados formatados e os converte para um arquivo JSON.
    
    - Se o arquivo JSON já existir, verifica se os novos dados já estão presentes.
    - Caso os dados já existam, não faz nenhuma alteração.
    - Caso os dados sejam novos, adiciona ao JSON existente.
    - Se o arquivo JSON não existir, cria um novo arquivo JSON com os dados extraídos.
    
    Parâmetros:
        - input_path (str): Caminho do arquivo de texto de entrada.
        - output_path (str): Caminho do arquivo JSON de saída.

    Exceções:
        - FileNotFoundError: Se o arquivo de entrada não for encontrado.
        - JSONDecodeError: Se o arquivo JSON existente estiver corrompido ou inválido.
        - IOError: Se houver erro ao escrever no arquivo de saída.
        - KeyError: Se os dados do JSON não contiverem as chaves esperadas ('datas').
        - TypeError: Se os dados lidos não estiverem no formato esperado.
    
    Retorna:
        dict: Estrutura de dados resultante contendo a contagem total e os dados extraídos.
    """
    
    try:
        with open(input_path, 'r', encoding='utf-8') as file:
            content = file.read().strip()

    except FileNotFoundError:
        print(f"Erro: Arquivo de entrada '{input_path}' não encontrado.")
        return {"count": 0, "datas": []}

    new_datas = []

    for block in filter(None, map(str.strip, content.split("###"))):
        parts = list(map(str.strip, block.split("---", 1)))
        if len(parts) == 2:
            text, texts_list = parts
            new_datas.append({"text": text, "texts": [t.strip() for t in texts_list.split("\n") if t.strip()]})

    if os.path.exists(output_path):
        try:
            with open(output_path, 'r', encoding='utf-8') as file:
                existing_data = json.load(file)

            if "datas" not in existing_data or not isinstance(existing_data["datas"], list):
                raise KeyError("O JSON existente não contém a chave 'datas' ou ela não está no formato esperado.")

        except (json.JSONDecodeError, FileNotFoundError):
            existing_data = {"count": 0, "datas": []}
        except KeyError as e:
            print(f"Erro: {e}")
            return {"count": 0, "datas": []}
        except TypeError as e:
            print(f"Erro: {e}")
            return {"count": 0, "datas": []}
    else:
        existing_data = {"count": 0, "datas": []}

    existing_texts = {(item["text"], tuple(item["texts"])) for item in existing_data["datas"]}
    added = 0

    for item in new_datas:
        if (item["text"], tuple(item["texts"])) not in existing_texts:
            existing_data["datas"].append(item)
            added += 1

    existing_data["count"] = len(existing_data["datas"])

    try:
        with open(output_path, 'w', encoding='utf-8') as output_file:
            json.dump(existing_data, output_file, indent=4, ensure_ascii=False)

    except IOError:
        print(f"Erro: Falha ao escrever no arquivo '{output_path}'.")
        return existing_data

    return existing_data

In [ ]:
input_path: str = "../helpers/txt_rase.txt"
output_path: str = "../databases/data_n1.json"

In [ ]:
parsed_data: dict = create_json_n1(input_path, output_path)
print(f"Processamento concluído. Resultado salvo em {output_path}")

Processamento concluído. Resultado salvo em ../databases/db_rase_n1.json
